Classification of Financial Stress Level Using Random Forest.

1. Importing Libraries

In [ ]:
from telnetlib import X3PAD

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nbformat
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn 
import csv
from sklearn.linear_model import LassoCV, Ridge
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder



2. Loading and Exploring Dataset

In [65]:
file_name='data_cleaned.csv'
df=pd.read_csv(file_name)
df.head()
non_numeric_cols = df.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_cols.tolist())

print(df[non_numeric_cols].head())




Non-numeric columns: ['Serial_No', 'Period', 'Job_Market_Difficulty', 'Good_Time_For_Large_Purchase', 'Future_Spend_On_Large_Purchases', 'Able_To_Meet_Regular_Expenses', 'Expected_Income_Change', 'Expected_Economic_Change_City', 'Expected_Economic_Change_Country', 'Earning_Difficulty', 'Country_Direction_Thoughts', 'Meals_Per_Day', 'Gender', 'Marital_Status', 'Age', 'Age_Group', 'Highest_Education_Level', 'Occupation', 'Population_Group', 'Children_Under_18', 'Household_Size', 'Total_Household_Income', 'Income_Level', 'Top_3_Concerns', 'Food_Beverages_Prices_Change', 'Utilities_Prices_Change', 'Furniture_Appliances_Prices_Change', 'Alcoholics_Prices_Change', 'Housing_Rent_Prices_Change', 'Transportation_Prices_Change', 'Clothing_Shoes_Prices_Change', 'Education_Prices_Change', 'Recreation_Prices_Change', 'Communication_Prices_Change', 'Concern_About_Rising_Prices', 'Rising_Prices_Impact_Day_To_Day_Expenses', 'Rising_Prices_Impact_Discretionary_Spending', 'Rising_Prices_Impact_Affordabl

3. Defining Target and Feature Selection

In [ ]:
y=df[['Stress_Level']]


ordinal_cols = [
    "Job_Market_Difficulty", "Future_Spend_On_Large_Purchases", "Able_To_Meet_Regular_Expenses",
    "Expected_Income_Change", "Expected_Economic_Change_City", "Expected_Economic_Change_Country",
    "Earning_Difficulty", "Income_Level", "Utilities_Prices_Change", "Alcoholics_Prices_Change",
    "Furniture_Appliances_Prices_Change", "Housing_Rent_Prices_Change", "Transportation_Prices_Change",
    "Clothing_Shoes_Prices_Change", "Education_Prices_Change", "Recreation_Prices_Change",
    "Communication_Prices_Change", "Concern_About_Rising_Prices",
    "Rising_Prices_Impact_Organic_Food", "Rising_Prices_Impact_Day_To_Day_Expenses",
    "Rising_Prices_Impact_Healthcare", "Age_Group","Marital_Status", "Household_Size"
]
# Define feature matrix
X = df[ordinal_cols].copy()
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[ordinal_cols] = encoder.fit_transform(X[ordinal_cols])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, max_depth=20,min_samples_leaf=1,min_samples_split=5,random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Print metrics
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

C:\Users\ggata\Documents\GitHub\Market-Analysis-Study\venv\Lib\site-packages\sklearn\base.py:1363: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Confusion Matrix:
[[ 70   8 120]
 [ 24  30  26]
 [ 34   8 467]]

Classification Report:
                 precision    recall  f1-score   support

        Neutral       0.55      0.35      0.43       198
  Not stressful       0.65      0.38      0.48        80
Quite stressful       0.76      0.92      0.83       509

       accuracy                           0.72       787
      macro avg       0.65      0.55      0.58       787
   weighted avg       0.70      0.72      0.69       787



4. Hyperparameter Tuning Using Grid Search

In [63]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train.values.ravel())

print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_



Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


In [53]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_cols = [
    "Job_Market_Difficulty", "Future_Spend_On_Large_Purchases", "Able_To_Meet_Regular_Expenses",
    "Expected_Income_Change", "Expected_Economic_Change_City", "Expected_Economic_Change_Country",
    "Earning_Difficulty", "Income_Level", "Utilities_Prices_Change", "Alcoholics_Prices_Change",
    "Furniture_Appliances_Prices_Change", "Housing_Rent_Prices_Change", "Transportation_Prices_Change",
    "Clothing_Shoes_Prices_Change", "Education_Prices_Change", "Recreation_Prices_Change",
    "Communication_Prices_Change", "Concern_About_Rising_Prices", 
    "Rising_Prices_Impact_Organic_Food", "Rising_Prices_Impact_Day_To_Day_Expenses",
    "Rising_Prices_Impact_Healthcare"
]

encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[ordinal_cols] = encoder.fit_transform(X[ordinal_cols])
